# Recommendation System



In [145]:
import pandas as pd
import numpy as np
import json
import re
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [146]:
spotify_df = pd.read_csv('data.csv')
spotify_df

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,1928,0.9950,0.708,0.1950,0.563000,0.1510,-12.428,0.0506,118.469,0.7790,1,10,0,0
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,1928,0.9940,0.379,0.0135,0.901000,0.0763,-28.454,0.0462,83.972,0.0767,1,8,0,0
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,1928,0.6040,0.749,0.2200,0.000000,0.1190,-19.924,0.9290,107.177,0.8800,0,5,0,0
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,9/25/28,1928,0.9950,0.781,0.1300,0.887000,0.1110,-14.734,0.0926,108.003,0.7200,0,1,0,0
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,1928,0.9900,0.210,0.2040,0.908000,0.0980,-16.829,0.0424,62.149,0.0693,1,11,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,4KppkflX7I3vJQk7urOJaS,Skechers (feat. Tyga) - Remix,"['DripReport', 'Tyga']",163800,5/15/20,2020,0.1730,0.875,0.4430,0.000032,0.0891,-7.461,0.1430,100.012,0.3060,1,1,75,1
169905,1ehhGlTvjtHo2e4xJFB0SZ,Sweeter (feat. Terrace Martin),"['Leon Bridges', 'Terrace Martin']",167468,6/8/20,2020,0.0167,0.719,0.3850,0.031300,0.1110,-10.907,0.0403,128.000,0.2700,1,8,64,0
169906,52eycxprLhK3lPcRLbQiVk,How Would I Know,"['Kygo', 'Oh Wonder']",180700,5/29/20,2020,0.5380,0.514,0.5390,0.002330,0.1080,-9.332,0.1050,123.700,0.1530,1,7,70,0
169907,3wYOGJYD31sLRmBgCvWxa4,I Found You,"['Cash Cash', 'Andy Grammer']",167308,2/28/20,2020,0.0714,0.646,0.7610,0.000000,0.2220,-2.557,0.0385,129.916,0.4720,1,1,70,0


In [147]:
spotify_df.head()

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,1928,0.995,0.708,0.1950,0.563,0.1510,-12.428,0.0506,118.469,0.7790,1,10,0,0
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,1928,0.994,0.379,0.0135,0.901,0.0763,-28.454,0.0462,83.972,0.0767,1,8,0,0
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,1928,0.604,0.749,0.2200,0.000,0.1190,-19.924,0.9290,107.177,0.8800,0,5,0,0
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,9/25/28,1928,0.995,0.781,0.1300,0.887,0.1110,-14.734,0.0926,108.003,0.7200,0,1,0,0
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,1928,0.990,0.210,0.2040,0.908,0.0980,-16.829,0.0424,62.149,0.0693,1,11,1,0


In [148]:
missing_values = spotify_df.isnull().sum()
missing_values[missing_values > 0]

,0


In [149]:
spotify_df.drop(["id", "release_date", "liveness"], axis=1, inplace=True)
spotify_df.head()

,name,artists,duration_ms,year,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,0.995,0.708,0.1950,0.563,-12.428,0.0506,118.469,0.7790,1,10,0,0
1,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,0.994,0.379,0.0135,0.901,-28.454,0.0462,83.972,0.0767,1,8,0,0
2,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,0.604,0.749,0.2200,0.000,-19.924,0.9290,107.177,0.8800,0,5,0,0
3,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,1928,0.995,0.781,0.1300,0.887,-14.734,0.0926,108.003,0.7200,0,1,0,0
4,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,0.990,0.210,0.2040,0.908,-16.829,0.0424,62.149,0.0693,1,11,1,0


# Handling Duplicates
Now let's check and remove duplicate data if there are any.

In [150]:
spotify_df[spotify_df.duplicated()]

,name,artists,duration_ms,year,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
609,"Rhapsody on a Theme of Paganini, Op.43: Variat...","['Sergei Rachmaninoff', 'William Kapell', 'Fri...",61467,1945,0.98300,0.635,0.0123,0.955000,-27.297,0.0521,83.046,0.610,1,0,0,0
652,Interview with William Kapell,['William Kapell'],1325093,1945,0.91300,0.704,0.2230,0.000000,-24.480,0.9350,112.485,0.403,0,2,0,0
3344,Give Me Your Love (Love Song),['Curtis Mayfield'],260333,1972,0.12800,0.591,0.6370,0.019500,-9.102,0.0887,172.452,0.962,0,5,26,0
8094,Enamórate De Alguien Más,['Morat'],230040,2020,0.46000,0.591,0.6590,0.000000,-5.464,0.0332,145.997,0.622,1,11,2,0
8169,"Danse macabre, S. 555 (after Camille Saint-Saens)","['Franz Liszt', 'Vladimir Horowitz']",491333,1928,0.99300,0.303,0.2260,0.826000,-21.483,0.0444,67.013,0.146,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164200,My Prayer,['Roy Orbison'],164467,1963,0.83800,0.493,0.2170,0.004630,-11.501,0.0271,105.003,0.182,1,9,20,0
166015,Bonzo's Montreux - 2012 Remaster,['Led Zeppelin'],262147,1982,0.21300,0.291,0.9750,0.908000,-10.754,0.0557,180.289,0.127,0,6,33,0
168041,Shiver,['Maroon 5'],179773,2002,0.04210,0.625,0.9250,0.000000,-4.435,0.3420,172.017,0.515,1,0,47,0
169035,Video Games - Remastered,['Lana Del Rey'],281947,2012,0.80600,0.390,0.2550,0.000001,-9.676,0.0299,122.056,0.179,0,6,50,0


In [151]:
spotify_df.drop_duplicates(inplace=True)
spotify_df

,name,artists,duration_ms,year,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,0.9950,0.708,0.1950,0.563000,-12.428,0.0506,118.469,0.7790,1,10,0,0
1,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,0.9940,0.379,0.0135,0.901000,-28.454,0.0462,83.972,0.0767,1,8,0,0
2,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,0.6040,0.749,0.2200,0.000000,-19.924,0.9290,107.177,0.8800,0,5,0,0
3,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,1928,0.9950,0.781,0.1300,0.887000,-14.734,0.0926,108.003,0.7200,0,1,0,0
4,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,0.9900,0.210,0.2040,0.908000,-16.829,0.0424,62.149,0.0693,1,11,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,Skechers (feat. Tyga) - Remix,"['DripReport', 'Tyga']",163800,2020,0.1730,0.875,0.4430,0.000032,-7.461,0.1430,100.012,0.3060,1,1,75,1
169905,Sweeter (feat. Terrace Martin),"['Leon Bridges', 'Terrace Martin']",167468,2020,0.0167,0.719,0.3850,0.031300,-10.907,0.0403,128.000,0.2700,1,8,64,0
169906,How Would I Know,"['Kygo', 'Oh Wonder']",180700,2020,0.5380,0.514,0.5390,0.002330,-9.332,0.1050,123.700,0.1530,1,7,70,0
169907,I Found You,"['Cash Cash', 'Andy Grammer']",167308,2020,0.0714,0.646,0.7610,0.000000,-2.557,0.0385,129.916,0.4720,1,1,70,0


We can observe that 607 rows were removed from the dataset

In [159]:
features = ['acousticness','danceability',	'energy',	'instrumentalness',	'speechiness',	'tempo',	'valence','duration_ms']
spotify_df[features]

,acousticness,danceability,energy,instrumentalness,speechiness,tempo,valence,duration_ms
0,0.9950,0.708,0.1950,0.563000,0.0506,118.469,0.7790,158648
1,0.9940,0.379,0.0135,0.901000,0.0462,83.972,0.0767,282133
2,0.6040,0.749,0.2200,0.000000,0.9290,107.177,0.8800,104300
3,0.9950,0.781,0.1300,0.887000,0.0926,108.003,0.7200,180760
4,0.9900,0.210,0.2040,0.908000,0.0424,62.149,0.0693,687733
...,...,...,...,...,...,...,...,...
169904,0.1730,0.875,0.4430,0.000032,0.1430,100.012,0.3060,163800
169905,0.0167,0.719,0.3850,0.031300,0.0403,128.000,0.2700,167468
169906,0.5380,0.514,0.5390,0.002330,0.1050,123.700,0.1530,180700
169907,0.0714,0.646,0.7610,0.000000,0.0385,129.916,0.4720,167308


In [115]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# spotify_df_scaled = scaler.fit_transform(spotify_df_features)
# spotify_df_scaled = pd.DataFrame(spotify_df_scaled, columns= features)

# Clustering songs into three clusters using KMeans algorithm

In [153]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(spotify_df[["acousticness", "danceability", "energy", "instrumentalness", "speechiness", "valence"]])
spotify_df["cluster"] = clusters
spotify_df.head()

,name,artists,duration_ms,year,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,cluster
0,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,0.995,0.708,0.1950,0.563,-12.428,0.0506,118.469,0.7790,1,10,0,0,2
1,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,0.994,0.379,0.0135,0.901,-28.454,0.0462,83.972,0.0767,1,8,0,0,2
2,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,0.604,0.749,0.2200,0.000,-19.924,0.9290,107.177,0.8800,0,5,0,0,1
3,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,1928,0.995,0.781,0.1300,0.887,-14.734,0.0926,108.003,0.7200,0,1,0,0,2
4,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,0.990,0.210,0.2040,0.908,-16.829,0.0424,62.149,0.0693,1,11,1,0,2


In [163]:
clustersfinal_df = spotify_df.groupby('cluster')[features].mean()
# clustersfinal_df["count"] = main_df.groupby("cluster").size()
# clustersfinal_df.insert(loc=0, column="count", value=spotify_df_features.groupby("cluster").size())
# clustersfinal_df[["count", "acousticness", "danceability", "energy", "instrumentalness", "speechiness", "tempo", "valence"]]
clustersfinal_df

,acousticness,danceability,energy,instrumentalness,speechiness,tempo,valence,duration_ms
cluster,,,,,,,,
0,0.148977,0.591042,0.689132,0.060588,0.100429,122.724516,0.599630,239068.284708
1,0.795690,0.516535,0.318393,0.030047,0.099748,113.199290,0.498045,213243.374008
2,0.877883,0.419748,0.250685,0.814595,0.059743,107.333468,0.397896,251088.488706


In [164]:
mood_mapping = {
    0: 'energetic',
    1: 'Relaxing',
    2: 'Acoustic Melancholy'
}
spotify_df['mood'] = spotify_df["cluster"].map(mood_mapping)
spotify_df

,name,artists,duration_ms,year,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,cluster,mood
0,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,0.9950,0.708,0.1950,0.563000,-12.428,0.0506,118.469,0.7790,1,10,0,0,2,Acoustic Melancholy
1,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,0.9940,0.379,0.0135,0.901000,-28.454,0.0462,83.972,0.0767,1,8,0,0,2,Acoustic Melancholy
2,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,0.6040,0.749,0.2200,0.000000,-19.924,0.9290,107.177,0.8800,0,5,0,0,1,Relaxing
3,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,1928,0.9950,0.781,0.1300,0.887000,-14.734,0.0926,108.003,0.7200,0,1,0,0,2,Acoustic Melancholy
4,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,0.9900,0.210,0.2040,0.908000,-16.829,0.0424,62.149,0.0693,1,11,1,0,2,Acoustic Melancholy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,Skechers (feat. Tyga) - Remix,"['DripReport', 'Tyga']",163800,2020,0.1730,0.875,0.4430,0.000032,-7.461,0.1430,100.012,0.3060,1,1,75,1,0,energetic
169905,Sweeter (feat. Terrace Martin),"['Leon Bridges', 'Terrace Martin']",167468,2020,0.0167,0.719,0.3850,0.031300,-10.907,0.0403,128.000,0.2700,1,8,64,0,0,energetic
169906,How Would I Know,"['Kygo', 'Oh Wonder']",180700,2020,0.5380,0.514,0.5390,0.002330,-9.332,0.1050,123.700,0.1530,1,7,70,0,1,Relaxing
169907,I Found You,"['Cash Cash', 'Andy Grammer']",167308,2020,0.0714,0.646,0.7610,0.000000,-2.557,0.0385,129.916,0.4720,1,1,70,0,0,energetic


# Using Spotify API

In [160]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import os

# Spotify API credentials
SPOTIPY_CLIENT_ID = 'your_client_id'
SPOTIPY_CLIENT_SECRET = 'your_client_secret'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

# Set up Spotify OAuth
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id='your_client_id',
    client_secret='your_client_secret'))


def find_song(name, year):
    song_data = defaultdict()
    results = sp.search(q='track: {} year: {}'.format(name, year), limit=1)
    if results['tracks']['items'] == []:
        return None

    result = results['tracks']['items'][0]
    track_id = result['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(result['explicit'])]
    song_data['duration_ms'] = [result['duration_ms']]
    song_data['popularity'] = [result['popularity']]
    for key, value in audio_features.items():
        song_data[key] = value

    columns = number_cols + ['name', 'year', 'explicit', 'duration_ms', 'popularity']
    for col in columns:
        if col not in song_data:
            song_data[col] = [None]

    return pd.DataFrame(song_data, columns=columns)




In [161]:
from collections import defaultdict
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']

def get_song_data(song, spotify_df):
    try:
        song_data = spotify_df[(spotify_df['name'] == song['name']) & (spotify_df['year'] == song['year'])].iloc[0]
        return song_data
    except IndexError:
        song_data = find_song(song['name'], song['year'])

        return song_data



def get_mean_vector(song_list, spotify_df):
    song_vectors = []

    for song in song_list:
        song_data = get_song_data(song, spotify_df)
        if song_data is None:
            print(f'Warning: {song["name"]} does not exist in Spotify or in database.')
            continue
        song_vector = song_data[number_cols].values

        if len(song_vector) != len(number_cols):
            print(f'Warning: {song["name"]} has an unexpected shape: {song_vector.shape}')
            continue

        song_vectors.append(song_vector)

    if not song_vectors:
        print('No valid song vectors found.')
        return None

    song_matrix = np.vstack(song_vectors)
    return np.mean(song_matrix, axis=0)



def flatten_dict_list(dict_list):

    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []

    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)

    return flattened_dict




In [162]:
def recommend_songs(song_list, spotify_df, n_songs=10):
    metadata_cols = ['name', 'year', 'artists']
    song_dict = flatten_dict_list(song_list)

    song_center = get_mean_vector(song_list, spotify_df)
    if song_center is None:
        print('No valid song vectors found. Cannot recommend songs.')
        return []

    input_song_clusters = []
    for song in song_list:
        song_data = get_song_data(song, spotify_df)
        if song_data is not None:
            if 'cluster' in song_data:
                input_song_clusters.append(song_data['cluster'])
            else:
                print(f'Warning: Song {song["name"]} does not have a cluster value.')

    if not input_song_clusters:
        print('No valid clusters found. Cannot filter songs.')
        return []
    unique_clusters = set(input_song_clusters)
    if 'cluster' not in spotify_df.columns:
        print('Error: The cluster column is missing from spotify_df.')
        return []

    filtered_data = spotify_df[spotify_df['cluster'].isin(unique_clusters)]

    raw_data = filtered_data[number_cols].values
    song_center = song_center.reshape(1, -1)
    distances = cdist(song_center, raw_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])

    rec_songs = filtered_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols].to_dict(orient='records')

# Example call to the function
recommend_songs([{'name': 'Videotape', 'year': 2007},
                 {'name': 'Smells Like Teen Spirit', 'year': 1991},
                 {'name': 'Wind Of Change', 'year': 1990},
                 {'name': 'O Children', 'year': 2004},
                 {'name': 'Do I Wanna Know?', 'year': 2013}], spotify_df)


[{'name': 'Se Va Muriendo Mi Alma',
  'year': 2014,
  'artists': "['Remmy Valenzuela']"},
 {'name': 'Vogue', 'year': 2009, 'artists': "['Madonna']"},
 {'name': 'Hells Bells', 'year': 1986, 'artists': "['AC/DC']"},
 {'name': 'Sad Girl', 'year': 2014, 'artists': "['Lana Del Rey']"},
 {'name': 'Lovesong', 'year': 2011, 'artists': "['Adele']"},
 {'name': 'Les', 'year': 2011, 'artists': "['Childish Gambino']"},
 {'name': 'Öyle Bir Yerdeyim ki',
  'year': 1986,
  'artists': "['Selda Bağcan', 'Ahmet Kaya']"},
 {'name': 'Man On The Moon', 'year': 1992, 'artists': "['R.E.M.']"},
 {'name': 'Estrelar', 'year': 1983, 'artists': "['Marcos Valle']"},
 {'name': 'Go Rest High On That Mountain',
  'year': 1994,
  'artists': "['Vince Gill']"}]